In [1]:
!pip install emoji autocorrect beautifulsoup4 nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 31.6 MB/s eta 0:00:00
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=0926cbd56037cd14b1408b12f7452fac7547a2c6361d18829ca1040eb18ab015
  Stored in directory: /root/.cache/pip/wheels/b6/28/c2/9ddf8f57f871b55b6fd0ab99c887531fb9a66e5ff236b82aee
Successfully built autocorrect


In [2]:
import pandas as pd
import re
import emoji
import string
import nltk

from bs4 import BeautifulSoup
from autocorrect import Speller
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import files
uploaded = files.upload()

Saving UNITENReview.csv to UNITENReview.csv


In [5]:
import os
print(os.listdir())

['.config', 'UNITENReview.csv', 'sample_data']


In [6]:
df = pd.read_csv("UNITENReview.csv")

In [7]:
pd.set_option('display.max_colwidth', None)
print(df.head())

                     Timestamp  \
0  2025/02/10 7:40:54 pm GMT+8   
1  2025/02/10 7:41:00 pm GMT+8   
2  2025/02/10 7:41:19 pm GMT+8   
3  2025/02/10 7:46:40 pm GMT+8   
4  2025/02/10 7:46:43 pm GMT+8   

                                                                                                                                                                                                                                                                                                                                                         Review  
0                                                                                                                                                                                                                                                                                                          Im happy with uniten actually, even the people are W  
1                                                                                      

In [8]:
spell = Speller(lang='en')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [9]:
slang_dict = {
    "tbh": "to be honest",
    "omg": "oh my god",
    "lol": "laugh out loud",
    "idk": "I don't know",
    "brb": "be right back",
    "btw": "by the way"
}

contractions_dict = {
    "can't": "cannot",
    "don't": "do not",
    "i'm": "i am",
    "it's": "it is"
}

In [10]:
def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', str(text))

def remove_html(text):
    return BeautifulSoup(str(text), "html.parser").get_text()

def remove_emojis(text):
    return emoji.replace_emoji(str(text), replace='')

def replace_slang(text):
    pattern = r'\b(' + '|'.join(map(re.escape, slang_dict.keys())) + r')\b'
    return re.sub(pattern,
                  lambda x: slang_dict[x.group().lower()],
                  str(text),
                  flags=re.IGNORECASE)

pattern_contr = re.compile(
    r'\b(' + '|'.join(map(re.escape, contractions_dict.keys())) + r')\b',
    flags=re.IGNORECASE
)

def replace_contractions(text):
    return pattern_contr.sub(
        lambda x: contractions_dict[x.group().lower()],
        str(text)
    )

def remove_punctuation(text):
    return str(text).translate(str.maketrans('', '', string.punctuation))

def remove_numbers(text):
    return re.sub(r'\d+', '', str(text))

def correct_spelling(text):
    return spell(str(text))

def remove_stopwords(text):
    words = str(text).split()
    return " ".join([w for w in words if w.lower() not in stop_words])

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text(text):
    words = word_tokenize(str(text))
    pos_tags = pos_tag(words)
    lemmas = [lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in pos_tags]
    return " ".join(lemmas)

In [11]:
def preprocess_text(text):
    text = str(text).lower()
    text = remove_urls(text)
    text = remove_html(text)
    text = remove_emojis(text)
    text = replace_slang(text)
    text = replace_contractions(text)
    text = remove_punctuation(text)
    text = remove_numbers(text)
    text = correct_spelling(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text

In [12]:
df["processed"] = df["Review"].apply(preprocess_text)
print(df[["Review", "processed"]].head())

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [13]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
df["processed"] = df["Review"].apply(preprocess_text)
print(df[["Review", "processed"]].head())

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [15]:
import nltk

nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [16]:
df["processed"] = df["Review"].apply(preprocess_text)
print(df[["Review", "processed"]].head())

                                                                                                                                                                                                                                                                                                                                                         Review  \
0                                                                                                                                                                                                                                                                                                          Im happy with uniten actually, even the people are W   
1                                                                                                                                                                                                                                                                                        I’m havin

In [17]:
df.to_csv("Processed_Reviews.csv", index=False)

In [18]:
files.download("Processed_Reviews.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>